<a href="https://colab.research.google.com/github/CristianCosci/LAB_MachineLearning_course/blob/main/CNN_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras.utils import np_utils

**The CIFAR-10 dataset**
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']


In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
print(train_images.shape)
print(train_images[0].shape)
#print(train_images[0])
# Normalize pixel values to be between 0 and 1

# Normalization
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
test_images =  test_images / 255.0

print("shape of train labels:", train_labels.shape)
print("some train labels",train_labels[:10])

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()


In [ ]:
# Create a CNN model  # add dropout and control overfitting # try to improve the accuracy
model = keras.Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(32, 32, 3))) 
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
# Need to pass from 2d data to 1d for the dense layer -> use a flatten layer
model.add(layers.Flatten())
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=len(class_names), activation='softmax'))

model.summary()



In [ ]:
# Compile and train
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

In [ ]:
# Save model
model.save('model_cnn.h5')

**Model evaluation**

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)

**Confusion matrix**

even if we use a different library to build our classifiers, we can still mix-and-match and use utilies functions from other libraries to analyze and evaluate the performance of a model. 

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(test_images)
matrix = confusion_matrix(test_labels, y_pred.argmax(axis=1))
print(matrix)

In [ ]:
import seaborn as sn
import pandas  as pd 
 
df_cm = pd.DataFrame(matrix, range(10),range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, cmap="BuPu",annot=True,annot_kws={"size": 10})# font size
plt.show()

**Analyze the outputs from intermediate convolutional layers**




In [ ]:
model.layers

In [ ]:
# Print the name and shape of the conv layers
# Summarize feature map shapes
for i in range(len(model.layers)):
	layer = model.layers[i]
	if 'conv' not in layer.name:
		continue
	
	print(i, layer.name, layer.output.shape)

In [ ]:
# Create a new model using layers from the previous model
# redefine model to output right after the first hidden layer
model_v = keras.Model(inputs = model.inputs, outputs= model.layers[0].output)
model_v.summary()

In [ ]:
# Get the feature maps for an images
feature_maps = model_v.predict(train_images[4].reshape(1, 32, 32, 3))
# print(feature_maps)
print(feature_maps.shape)

In [ ]:
plt.imshow(train_images[4])

In [ ]:
# Plot the feature maps
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
fig  = plt.figure(figsize=(12,12))

for i in range(32):
    sub = fig.add_subplot(8,4, i+1)
    sub.imshow(feature_maps[0,:,:,i], cmap = "gray") 
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

In [ ]:
# Repeat the above process using the second conv layer in our initial model
# redefine model to output right after the second conv  layer
model_v_2 = keras.Model(inputs = model.inputs, outputs= model.layers[2].output)
model_v_2.summary()

In [ ]:
# Get the feature maps for an images
feature_maps_2 = model_v_2.predict(train_images[4].reshape(1, 32, 32, 3))
#print(feature_maps_2)
print(feature_maps_2.shape)

In [ ]:
# Plot the featue maps
fig  = plt.figure(figsize=(12,12))

for i in range(64):
    sub = fig.add_subplot(8,8, i+1)
    sub.imshow(feature_maps_2[0,:,:,i], cmap = "gray")
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

In [ ]:
# Repeat the above process using the third conv layer in our initial model
# redefine model to output right after the third conv  layer
model_v_4 = keras.Model(inputs = model.inputs, outputs= model.layers[4].output)
#TO DO: get the feature maps for an images
feature_maps_4 = model_v_4.predict(train_images[4].reshape(1, 32, 32, 3))
#print(feature_maps_2)
print(feature_maps_4.shape)

#TO DO : plot the featue maps
fig  = plt.figure(figsize=(12,12))

for i in range(64):
    sub = fig.add_subplot(8,8, i+1)
    sub.imshow(feature_maps_4[0,:,:,i], cmap = "gray")
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

**HOMEWORK:  plot the learned filters**

Hint: the learned filters are simply the weights